In [ ]:
#Import the necessary libraries

In [ ]:
from datetime import timedelta
from stat import ST_DEV
from sys import argv
import numpy as np
import pandas as pd
import time
import math
import os
import sklearn
import matplotlib.pyplot as plt

In [ ]:
# clear the system

In [ ]:
clear = lambda: os.system('clear')

In [ ]:
#The main st-dbscan aşgorithm part

In [ ]:
def ST_DBSCAN(df, Eps1, Eps2, min):#takes data,temporal treshold,spatial treshold and minimum number of neighbors as variables
    cluster = 0
    stack = []
    outlier = -1                   #outliers are marked with -1
    unmarked = -2
    df['cluster'] = unmarked
    for key, value in df.iterrows():
        if df.loc[key]['cluster'] == unmarked:  # mark as no mark
            x = neighbors(df,key, Eps1, Eps2)   #retrieve neighbors
            if len(x) < min:
                df.at[key, 'cluster'] = outlier  # mark as outlier
            else:  # new cluster
                cluster = cluster + 1
                df.at[key, 'cluster'] = cluster   #assign new core label to cluster
                for i in range(len(x)):
                    df.at[i, 'cluster'] = cluster
                    stack.append(i)
                while not stack.IsEmpty():
                    CurrentObj = stack.pop()
                    y = neighbors(df,CurrentObj, Eps1, Eps2)    #find the neighbors new core
                    if len(y) >= min:              
                        for index2 in y:
                            if (df.loc[index2]['cluster'] != outlier) or (df.loc[index2]['cluster'] == unmarked):
                                # if len(cluster_avg()-nei_i)<=e)
                                df.at[index2, 'cluster'] = cluster              #mark current data as cluster
                                stack.append(index2)
    return df

In [ ]:
#the function that finds the neighbors

In [ ]:
def neighbors(df, obj, eps1, eps2):  # takes all data,index of the velue which neighbors are wanted,temporal treshold,spatial treshold as variables
    neighbor = []
    center = df.loc[obj]

    min_time = center['date_time'] - timedelta(minutes=eps2)    #filter data according to time with temporal treshold
    max_time = center['date_time'] + timedelta(minutes=eps2)
    df = df[(df['date_time'] >= min_time) & (df['date_time'] <= max_time)]

    for keys, val in df.iterrows():                             #filter data according to distance with spatial treshold
        if keys == obj:
            continue
        else:
            distance = dist_calc(center['latitude'], center['longitude'], val['latitude'], val['longitude'])
            if distance <= eps1:
                neighbor.append(keys)

    return neighbor


In [ ]:
#the function that finds the distance between two latitude and longitudes 

In [ ]:
def dist_calc(lat1, lon1, lat2, lon2):
    radius = 6371                      #km radius
    dlat = math.radians(lat2 - lat1)
    dlon = math.radians(lon2 - lon1)
    a = (math.sin(dlat / 2) * math.sin(dlat / 2) +
         math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) *
         math.sin(dlon / 2) * math.sin(dlon / 2))
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    dist = radius * c
    return dist

In [ ]:
#the part that plots the clusters

In [ ]:
def plot_clusters(df, output_name):
	labels = df['cluster'].values
    X = df[['longitude', 'latitude']].values
	unique_labels = set(labels)
    colors = [plt.cm.Spectral(each)
              for each in np.linspace(0, 1, len(unique_labels))]
    for k, col in zip(unique_labels, colors):
        if k == -1:
            col = [0, 0, 0, 1]

        class_member_mask = (labels == k)

        xy = X[class_member_mask]
        plt.plot(xy[:, 0], xy[:, 1], 'o', markerfacecolor=tuple(col),
                 markeredgecolor='k', markersize=6)

    plt.show()

In [ ]:
#main funtion of python

In [ ]:
data = pd.read_csv("sample.csv", sep=",") # read the data
sthreshold=50
tthreshold=10
mneighbors=2
print(data)  # print the data table
st_dbscan =ST_DBSCAN(data,sthreshold,tthreshold,mneighbors)